# Does the local gateway help the fullnode to receive blocks faster?

This notebook compares blocks receiving speed between with and without gateways.

Steps:

* First, start a fullnode with a local bloXroute gateway, and run the [bloxroute_block](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/bloxroute/block/bloxroute_block.go) and [fullnode_block](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/fullnode/block/fullnode_block.go) to collect data
* Second, stop the local gateway, and run `bloxroute_block` and `fullnode_block` to collect data again
* Substract the `received_at` timestamp of blocks with the same hash, we get to know the gap between with and without gateway.

Note: Make sure the BSC fullnode is fully synced before this benchmark, type `eth.syncing` in geth console and make sure it is `faluse`, which means the fullnode is synced.

Hardware:

* A machine with 128 cores, 256GB memory in a Fremont IDC
* A `m5zn.3xlarge` instance in AWS Virginia regon

Files:

* `bloxroute-block-cloud-fremont-with-gateway.json.gz`, `bloxroute-block-cloud-virginia-with-gateway.json.gz`, `fullnode-block-fremont-with-gateway.json.gz` and `fullnode-block-virginia-with-gateway.json.gz` are generated in step 1
* `bloxroute-block-cloud-fremont-without-gateway.json.gz`, `bloxroute-block-cloud-virginia-without-gateway.json.gz`, `fullnode-block-fremont-without-gateway.json.gz` and `fullnode-block-virginia-without-gateway.json.gz` are generated in step 2

In [1]:
import gzip
import json
import pandas as pd
from typing import Dict

In [2]:
pd.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

In [3]:
def read_timestamp(file: str) -> Dict[str, int]:
    result: Dict[str, int] = {}  # hash -> received_at
    f_in = gzip.open(file, "rt") if file.endswith('.json.gz') else open(file, "rt")
    with f_in:
        for line in f_in:
            obj = json.loads(line)
            if 'received_at' in obj and 'hash' in obj:
                result[obj['hash']] = obj['received_at']
    return result

In [4]:
def compare_timestamp(file1: str, file2: str) -> pd.DataFrame:
    dict1 = read_timestamp(file1)
    dict2 = read_timestamp(file2)
    diff = {}
    for tx_hash, received_at in dict1.items():
        if tx_hash in dict2:
            diff[tx_hash] = received_at-dict2[tx_hash]
    
    df = pd.DataFrame(diff.items(), columns=['tx_hash', 'gap'])
    # remove outliers
    quantile05 = df['gap'].quantile(0.05)
    quantile95 = df['gap'].quantile(0.95)
    return df[(df['gap'] >= quantile05) & (df['gap']<=quantile95)]

## In Fremont IDC

In [5]:
compare_timestamp('./data/bloxroute-block-cloud-fremont-without-gateway.json.gz', './data/fullnode-block-fremont-without-gateway.json.gz').describe()

,gap
count,7614.000000
mean,-951.281849
std,337.746817
min,-1600.000000
25%,-1145.000000
50%,-1016.500000
75%,-840.000000
max,-129.000000


In [6]:
compare_timestamp('./data/bloxroute-block-cloud-fremont-with-gateway.json.gz', './data/fullnode-block-fremont-with-gateway.json.gz').describe()

,gap
count,8065.000000
mean,-192.666460
std,111.773782
min,-692.000000
25%,-228.000000
50%,-160.000000
75%,-119.000000
max,-52.000000


**Conclusion: Without a local gateway the gap between the fullnode and bloXroute cloud is 1016ms, while with a local gateway the gap reduces to 160ms, so the local gateway obviously helps the local fullnode receive blocks faster!**

**Conclusion: The cloud API is 1016ms faster than the fullnode without gateway, and 160ms faster than the fullnode with a local gateway, which means the fullnode is 856ms faster after a local gateway is running.**

## In AWS Virginia

In [7]:
compare_timestamp('./data/bloxroute-block-cloud-virginia-without-gateway.json.gz', './data/fullnode-block-virginia-without-gateway.json.gz').describe()

,gap
count,7602.000000
mean,-443.515917
std,287.117700
min,-1467.000000
25%,-534.000000
50%,-347.000000
75%,-243.000000
max,-131.000000


In [8]:
compare_timestamp('./data/bloxroute-block-cloud-virginia-with-gateway.json.gz', './data/fullnode-block-virginia-with-gateway.json.gz').describe()

,gap
count,8066.000000
mean,-424.814034
std,262.458800
min,-1511.000000
25%,-506.000000
50%,-344.000000
75%,-246.000000
max,-139.000000


**Conclusion: The cloud API is 347ms faster than the fullnode without gateway, and 344ms faster than the fullnode with a local gateway, which means the fullnode is 8ms faster after a local gateway is running.**

## Compare two fullnodes

In [9]:
compare_timestamp('./data/fullnode-block-virginia-without-gateway.json.gz', './data/fullnode-block-fremont-without-gateway.json.gz').describe()

,gap
count,7595.000000
mean,-508.390388
std,405.706073
min,-1185.000000
25%,-818.000000
50%,-632.000000
75%,-228.000000
max,561.000000


In [10]:
compare_timestamp('./data/fullnode-block-virginia-with-gateway.json.gz', './data/fullnode-block-fremont-with-gateway.json.gz').describe()

,gap
count,8125.000000
mean,200.232123
std,197.873855
min,-68.000000
25%,71.000000
50%,144.000000
75%,261.000000
max,1062.000000


**Conclusion: Without local gateways, the fullnode in Fremont is 632ms slower than the fullnode in AWS virginia, after a local gateway is up and running, the Fremont fullnode is 144ms faster than the Virginia one!**

## Conclusion

**Overall, this benchmark shows that the local gateway DOES help the local fullnode receive blocks faster, but if your fullnode is syncing fast enough, the local gateway can NOT help much.**